In [54]:
import requests
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import time
from lxml.html import fromstring
from collections import defaultdict

In [50]:
url = 'https://www.cian.ru/sale/flat/227051138/'

r = requests.get(url)
content = r.content
soup = BeautifulSoup(content)

In [75]:
def get_building_info(soup):
    desc = soup.find_all('div', {'class': 'a10a3f92e9--info-title--2bXM9'})
    info = soup.find_all('div', {'class' : 'a10a3f92e9--info-value--18c8R'})
    return desc, info

In [90]:
desc, info = get_building_info(soup)
desc
info

[<div class="a10a3f92e9--info-value--18c8R">250,9 м²</div>,
 <div class="a10a3f92e9--info-value--18c8R">138,6 м²</div>,
 <div class="a10a3f92e9--info-value--18c8R">19,8 м²</div>,
 <div class="a10a3f92e9--info-value--18c8R">10 из 13</div>,
 <div class="a10a3f92e9--info-value--18c8R">4 кв. 2019</div>]

In [101]:
def to_info(desc, info):
    text = { }
    for k,v in zip(desc, info):
        text[k.text] = v.text
    return(text)

In [102]:
full_info = to_info(desc, info)
full_info

{'Общая': '250,9\xa0м²',
 'Жилая': '138,6\xa0м²',
 'Кухня': '19,8\xa0м²',
 'Этаж': '10 из 13',
 'Срок сдачи': '4 кв. 2019'}